# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Initialize-Environment" data-toc-modified-id="Initialize-Environment-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev1 toc-item"><a href="#Generate-list-of-data" data-toc-modified-id="Generate-list-of-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Generate list of data</a></div><div class="lev2 toc-item"><a href="#Load-Data" data-toc-modified-id="Load-Data-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Load Data</a></div><div class="lev1 toc-item"><a href="#!!!-NMF-TEST-!!!" data-toc-modified-id="!!!-NMF-TEST-!!!-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>!!! NMF TEST !!!</a></div><div class="lev1 toc-item"><a href="#Rank-Subgraphs-Based-on-Pos/Neg-Expression" data-toc-modified-id="Rank-Subgraphs-Based-on-Pos/Neg-Expression-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Rank Subgraphs Based on Pos/Neg Expression</a></div><div class="lev1 toc-item"><a href="#Subgraph-Motion-Detection" data-toc-modified-id="Subgraph-Motion-Detection-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Subgraph Motion-Detection</a></div><div class="lev1 toc-item"><a href="#Plot-Ranked-System-Subgraph-Matrices" data-toc-modified-id="Plot-Ranked-System-Subgraph-Matrices-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Plot Ranked System Subgraph Matrices</a></div><div class="lev1 toc-item"><a href="#Expression-Constrasts" data-toc-modified-id="Expression-Constrasts-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Expression Constrasts</a></div><div class="lev2 toc-item"><a href="#Stroop-vs-Navon" data-toc-modified-id="Stroop-vs-Navon-71"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Stroop vs Navon</a></div><div class="lev3 toc-item"><a href="#Expression-Plot" data-toc-modified-id="Expression-Plot-711"><span class="toc-item-num">7.1.1&nbsp;&nbsp;</span>Expression Plot</a></div><div class="lev2 toc-item"><a href="#Lo-vs-Hi" data-toc-modified-id="Lo-vs-Hi-72"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Lo vs Hi</a></div><div class="lev3 toc-item"><a href="#Expression-Plot" data-toc-modified-id="Expression-Plot-721"><span class="toc-item-num">7.2.1&nbsp;&nbsp;</span>Expression Plot</a></div><div class="lev3 toc-item"><a href="#Render-t-stat-map" data-toc-modified-id="Render-t-stat-map-722"><span class="toc-item-num">7.2.2&nbsp;&nbsp;</span>Render t-stat map</a></div><div class="lev1 toc-item"><a href="#Subgraph-Regime-Switching" data-toc-modified-id="Subgraph-Regime-Switching-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Subgraph Regime Switching</a></div><div class="lev1 toc-item"><a href="#Task-Performance" data-toc-modified-id="Task-Performance-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Task Performance</a></div><div class="lev2 toc-item"><a href="#Behavioral-Correlation" data-toc-modified-id="Behavioral-Correlation-91"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Behavioral Correlation</a></div><div class="lev2 toc-item"><a href="#Regional-Contribution-to-Modulation-in-Performance" data-toc-modified-id="Regional-Contribution-to-Modulation-in-Performance-92"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>Regional Contribution to Modulation in Performance</a></div>

# Initialize Environment

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %reset
except:
    print 'NOT IPYTHON'

from __future__ import division

import os
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['NUMEXPR_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '1'
import sys
import glob

import numpy as np
import pandas as pd
#import seaborn as sns
import scipy.stats as stats
import scipy.io as io
import h5py
import matplotlib
import matplotlib.pyplot as plt

sys.path.append('/Users/akhambhati/Developer/hoth_research/Echobase')
import Echobase
convert_conn_vec_to_adj_matr = Echobase.Network.Transforms.configuration.convert_conn_vec_to_adj_matr
convert_adj_matr_to_cfg_matr = Echobase.Network.Transforms.configuration.convert_adj_matr_to_cfg_matr

rcParams = Echobase.Plotting.fig_format.update_rcparams(matplotlib.rcParams)

path_Remotes = '/Users/akhambhati/Remotes'
path_CoreData = path_Remotes + '/CORE.fMRI_cogcontrol.medaglia'
path_PeriphData = path_Remotes + '/RSRCH.NMF_CogControl'
path_InpData_Netw = path_PeriphData + '/e01-FuncNetw'
path_InpData_Subg = path_PeriphData + '/e02-FuncSubg'
path_ExpData = path_PeriphData + '/e03-FuncSubg_Dynamics'
path_Figures = './e03-Figures/'

for path in [path_CoreData, path_PeriphData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)

# Generate list of data

## Load Data

In [ ]:
%matplotlib inline

# Load Functional Data
df_cfg = np.load('{}/Population.Configuration_Matrix.npz'.format(path_InpData_Netw))
df_subgraph = np.load('{}/Yeo_Subgraph.All.npz'.format(path_InpData_Subg))
df_to_yeo = np.load('{}/Lausanne125_to_Yeo.npz'.format(path_InpData_Netw))

n_subj, _, _, _, _, n_block = df_cfg['cfg_obs_lut'].shape
n_fac = len(df_subgraph['system_subgraph'])


# Load Behavioral Data
df_blk = io.loadmat('{}/BlockwiseDataCorrectTrialsOnly.mat'.format(path_CoreData))
bad_subj_ix = [1, 6]
good_subj_ix = np.setdiff1d(np.arange(n_subj+2), bad_subj_ix)
"""
'high control accuracy', 'low control accuracy', 'high control mean RT',
'high control median RT', 'low control mean RT', 'low control median RT'
"""
df_perf = {'Stroop': {'lo': {'accuracy': df_blk['StroopData'][good_subj_ix, 1, :],
                             'meanRT': df_blk['StroopData'][good_subj_ix, 4, :],
                             'medianRT': df_blk['StroopData'][good_subj_ix, 5, :]},
                      'hi': {'accuracy': df_blk['StroopData'][good_subj_ix, 0, :],
                             'meanRT': df_blk['StroopData'][good_subj_ix, 2, :],
                             'medianRT': df_blk['StroopData'][good_subj_ix, 3, :]}
                     },
           'Navon' : {'lo': {'accuracy': df_blk['NavonData'][good_subj_ix, 1, :],
                             'meanRT': df_blk['NavonData'][good_subj_ix, 4, :],
                             'medianRT': df_blk['NavonData'][good_subj_ix, 5, :]},
                      'hi': {'accuracy': df_blk['NavonData'][good_subj_ix, 0, :],
                             'meanRT': df_blk['NavonData'][good_subj_ix, 2, :],
                             'medianRT': df_blk['NavonData'][good_subj_ix, 3, :]}
                     }
          }


# Load Motion Data
df_motion = {'Stroop': io.loadmat('{}/StroopMove.mat'.format(path_CoreData))['move'][:, 0],
             'Navon': io.loadmat('{}/NavonMove.mat'.format(path_CoreData))['move'][:, 0]}

# !!! NMF TEST !!!

In [ ]:
cfg_matr = df_cfg['cfg_matr'].copy()
#cfg_matr = (df_cfg['cfg_matr'].T * df_cfg['cfg_L2']).T

rank = 13
alpha = 1e-7
beta = 1e-7
n_fac = rank

# Grab the task ID of the current job (and the associated parameter dictionary)
fac_subnet = np.random.uniform(low=0, high=1.0, size=(rank, cfg_matr.shape[1]))
fac_coef = np.random.uniform(low=0, high=1.0, size=(rank, cfg_matr.shape[0]))

# Run NMF Algorithm
fac_subnet, fac_coef, err = Echobase.Network.Partitioning.Subgraph.nmf.snmf_bcd(
    cfg_matr, alpha=alpha, beta=beta, fac_subnet_init=fac_subnet, fac_coef_init=fac_coef, 
    max_iter=20, sparse_dim='conn', verbose=True)

# Rank Subgraphs Based on Pos/Neg Expression

In [ ]:
import string
abcd = list(string.ascii_uppercase)

coef_ix = np.array(df_cfg['cfg_obs_lut'], dtype=int)

# Re-rank subgraphs based on positive/negative expression
del_expr_mean = []
del_expr_stdv = []
for fac_ii in xrange(n_fac):
    #sel_fac_coef = df_subgraph['system_subgraph'][fac_ii]['expr_coef'][coef_ix]
    sel_fac_coef = fac_coef[fac_ii, :][coef_ix]
    pos_expr = sel_fac_coef[:, :, :, :, 0, :]
    neg_expr = sel_fac_coef[:, :, :, :, 1, :]
    del_expr = (pos_expr-neg_expr).mean(axis=-1).mean(axis=-1).mean(axis=-1).mean(axis=-1)
    
    del_expr_mean.append(del_expr.mean())
    del_expr_stdv.append(del_expr.std() / np.sqrt(n_subj))    
del_expr_mean = np.array(del_expr_mean)
del_expr_stdv = np.array(del_expr_stdv)
sort_fac = np.argsort(del_expr_mean)[::-1]

# Create a sorted dictionary
sort_fac_dict = {}
for ltr, fac_ii in zip(abcd, sort_fac):
    sort_fac_dict[ltr] = fac_ii


# Plot distribution of mean relative expression
plt.figure(dpi=300)
ax = plt.subplot(111)
ax.bar(xrange(n_fac), del_expr_mean[sort_fac], yerr=del_expr_stdv[sort_fac], lw=0)

ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

ax.set_xticks(np.arange(n_fac)+0.4);
ax.set_xticklabels(np.sort(sort_fac_dict.keys()));

ax.set_xlim([0, n_fac])

ax.set_xlabel('Subgraphs')
ax.set_ylabel('Mean Relative Expression')

plt.show()

# Subgraph Motion-Detection

In [ ]:
alpha = 0.05 / n_fac

motion_rv = []
motion_color = []
motion_fac = []
coef_ix = np.array(df_cfg['cfg_obs_lut'], dtype=int)
for fac_key in np.sort(sort_fac_dict.keys()):
    fac_ii = sort_fac_dict[fac_key]
    #sel_fac_coef = df_subgraph['system_subgraph'][fac_ii]['expr_coef'][coef_ix]
    sel_fac_coef = fac_coef[fac_ii, :][coef_ix]    
    fac_expr_subj = sel_fac_coef.reshape(n_subj, -1).mean(axis=-1)

    rv, pv = stats.pearsonr(0.5*(df_motion['Stroop'] + df_motion['Navon']),
                            fac_expr_subj)
    motion_rv.append(rv)
    
    if pv < alpha:
        motion_color.append('r')
        motion_fac.append(fac_key)
    else:
        motion_color.append('k')
        
motion_rv = np.array(motion_rv)
motion_color = np.array(motion_color)
motion_fac = np.array(motion_fac)


# Identify Subgraphs that correlate with motion
plt.figure(dpi=300)
ax = plt.subplot(111)
ax.bar(xrange(n_fac), motion_rv,
       lw=0, color=motion_color)

ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

ax.set_xticks(np.arange(n_fac)+0.4);
ax.set_xticklabels(np.sort(sort_fac_dict.keys()));

ax.set_ylim([-1, 1])
ax.set_xlim([0, n_fac])

ax.set_xlabel('Subgraphs')
ax.set_ylabel('PearsonR(Expression, Motion)')

plt.show()

# Plot Ranked System Subgraph Matrices

In [ ]:
%matplotlib inline

n_laus = len(df_to_yeo['yeo_lbl'])
n_yeo = len(df_to_yeo['yeo_name'])

fsize = 5.5
fig = plt.figure(figsize=(6.9, 6.9), dpi=300)
for ii, fac_key in enumerate(np.sort(sort_fac_dict.keys())):
    if fac_key in motion_fac:
        continue        
    fac_ii = sort_fac_dict[fac_key]
    
    vmin, vmax = fac_subnet[fac_ii, :].min(), fac_subnet[fac_ii, :].max()
    
    ax = fig.add_subplot(5, 5, ii+1)
    #sel_fac_subnet = df_subgraph['system_subgraph'][fac_ii]['subnet_roi']
    sel_fac_subnet = convert_conn_vec_to_adj_matr(fac_subnet[fac_ii, :])[df_to_yeo['sort_laus_to_yeo'], :][:, df_to_yeo['sort_laus_to_yeo']]
    mat = ax.matshow(sel_fac_subnet, cmap='magma', vmin=vmin, vmax=vmax)
    
    for xx in df_to_yeo['yeo_adj_demarc']:
        ax.vlines(xx, 0, n_laus, color='w', lw=0.25)
        ax.hlines(xx, 0, n_laus, color='w', lw=0.25)

    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_tick_params(width=0)                                
    ax.xaxis.set_tick_params(width=0)
    ax.grid(False)
    ax.tick_params(axis='both', which='major', pad=-3)

    ax.set_xticks((df_to_yeo['yeo_adj_demarc'][:-1] + (np.diff(df_to_yeo['yeo_adj_demarc']) * 0.5)));
    ax.set_xticklabels(df_to_yeo['yeo_name'], fontsize=5.0, rotation=45)

    ax.set_yticks((df_to_yeo['yeo_adj_demarc'][:-1] + (np.diff(df_to_yeo['yeo_adj_demarc']) * 0.5)));
    ax.set_yticklabels(df_to_yeo['yeo_name'], fontsize=5.0, rotation=45)
    
fig.tight_layout(pad=0.01, h_pad=0.001, w_pad=0.001)
fig.savefig('{}/Ranked_System_Subgraph.svg'.format(path_Figures))
fig.show()

# Expression Constrasts

## Stroop vs Navon

### Expression Plot

In [ ]:
%matplotlib inline

coef_ix = np.array(df_cfg['cfg_obs_lut'], dtype=int)
alpha = 0.05 #/ n_fac
pv = []

plt.figure(figsize=(4,6), dpi=300)
for ii, fac_key in enumerate(np.sort(sort_fac_dict.keys())):
    if fac_key in motion_fac:
        continue        
    fac_ii = sort_fac_dict[fac_key]

    #sel_fac_coef = df_subgraph['system_subgraph'][fac_ii]['expr_coef'][coef_ix]
    sel_fac_coef = fac_coef[fac_ii, :][coef_ix]    
    pos_expr = sel_fac_coef[:, :, :, :, 0, :]
    neg_expr = sel_fac_coef[:, :, :, :, 1, :]
    
    del_expr = (pos_expr-neg_expr).mean(axis=-1)[:, :, :, 1].mean(axis=-1)
    del_expr_mean = del_expr.mean(axis=0)
    del_expr_stdv = del_expr.std(axis=0) / np.sqrt(n_subj)    

    # Within-experiment Stats
    tv_expr, pv_expr = stats.ttest_rel(*del_expr.T)
    pv.append(pv_expr)
    
    ax = plt.subplot(5,5,ii+1)
    ax.bar([0,1], del_expr_mean,
           yerr=del_expr_stdv,
           width=0.4, lw=0)
    
    #ax.text(0.5, -4, 't={:2.2f}\np={:2.1e}'.format(tv_expr, pv_expr), fontsize=3.0)
    if pv_expr < alpha:
        print(fac_key, '*')
        #ax.text(0.65, -5, '*', fontsize=3.0)
    
    ax.set_title(fac_key)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_xticks(np.array([0, 1]) + 0.2)
    ax.set_xticklabels(df_cfg['cfg_key_label'][()]['Experiment_ID'])
    #ax.set_ylim([-6.5, 6.5])

#fig.savefig('{}/Ranked_System_Subgraph.svg'.format(path_Figures))
plt.show()
#plt.close()

## Lo vs Hi

### Expression Plot

In [ ]:
%matplotlib inline

coef_ix = np.array(df_cfg['cfg_obs_lut'], dtype=int)
alpha = 0.05 #/ n_fac

color = [np.array([95, 127, 237]) / 255,
         np.array([218, 85, 65]) / 255]

plt.figure(figsize=(4,6), dpi=300)
for ii, fac_key in enumerate(np.sort(sort_fac_dict.keys())):
    if fac_key in motion_fac:
        continue        
    fac_ii = sort_fac_dict[fac_key]

    ax = plt.subplot(5,5,ii+1)
    
    for expr_ii, expr_id in enumerate(df_cfg['cfg_key_label'][()]['Experiment_ID']):
        #sel_fac_coef = df_subgraph['system_subgraph'][fac_ii]['expr_coef'][coef_ix]
        sel_fac_coef = fac_coef[fac_ii, :][coef_ix]
        pos_expr = sel_fac_coef[:, expr_ii, :, :, :, :][:, :, :, 0, :][:, :, 1, :]
        neg_expr = sel_fac_coef[:, expr_ii, :, :, :, :][:, :, :, 1, :][:, :, 1, :]

        del_expr = (pos_expr-neg_expr).mean(axis=-1)
        del_expr_mean = del_expr.mean(axis=0)
        del_expr_stdv = del_expr.std(axis=0) / np.sqrt(n_subj)    

        # Within-experiment Stats
        tv_expr, pv_expr = stats.ttest_rel(*del_expr.T)
        if pv_expr < alpha:
            clr = color[expr_ii]
        else:
            clr = [0.5, 0.5, 0.5]
        

        ax.bar([3*expr_ii+0, 3*expr_ii+1], del_expr_mean,
               yerr=del_expr_stdv, color=clr,
               width=0.4, lw=0)
    #ax.text(0, 7, df_cfg['cfg_key_label'][()]['Experiment_ID'][0], fontsize=4.0)
    #ax.text(3, 7, df_cfg['cfg_key_label'][()]['Experiment_ID'][1], fontsize=4.0)

    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_xticks(np.array([0, 1, 3, 4]) + 0.2)
    ax.set_xticklabels(np.concatenate((df_cfg['cfg_key_label'][()]['Condition_ID'],
                                       df_cfg['cfg_key_label'][()]['Condition_ID'])))
    #ax.set_ylim([-6.5, 6.5])

#fig.savefig('{}/Ranked_System_Subgraph.svg'.format(path_Figures))
plt.show()
#plt.close()

### Render t-stat map

In [ ]:
from mayavi import mlab
import nibabel as nib

roi_ptcp = np.array([np.sum(tv[ii]*df_subgraph['system_subgraph'][fac_ii]['subnet_roi'], axis=0)
                     for ii, fac_ii in enumerate(sort_fac)]).sum(axis=0)
xmax = 10 #np.max([np.abs(roi_ptcp.min()), roi_ptcp.max()])

brain_system_pixmap = {}
view_angle = {'Sag_PA': [0.0, 90.0],
              'Sag_AP': [180.0, 90.0]}

# Get the pial surface recons
pial_hemi = {'LH': {},
             'RH': {}}
pial_hemi['LH']['vert'], pial_hemi['LH']['tria'] = nib.freesurfer.io.read_geometry('{}/BrainRenderSubject15/surf/lh.pial'.format(path_CoreData))
pial_hemi['RH']['vert'], pial_hemi['RH']['tria'] = nib.freesurfer.io.read_geometry('{}/BrainRenderSubject15/surf/rh.pial'.format(path_CoreData))

# Get the Lausanne label files for each ROI
label_files = []
for roi in df_to_yeo['df_laus_yeo']:
    laus_lbl = roi[1].lower()
    hemi = roi[2].lower()

    # Parse the atlas name and find the label file if it exists
    lbl_file = '{}.{}.label'.format(hemi, laus_lbl)
    lbl_file = lbl_file.replace(' ', '')

    label_files.append('{}/BrainRenderSubject15/label/regenerated_{}_125/{}'.format(path_CoreData, hemi, lbl_file))

# Iterate over hemisphere of the pial surface
for hemi in pial_hemi.keys():
    n_vert = len(pial_hemi[hemi]['vert'])
    pial_scalars = np.ones(n_vert)

    # Iterate over brain regions
    for roi_ix, (roi, lbl_file) in enumerate(zip(df_to_yeo['df_laus_yeo'], label_files)):
        if roi[2] != hemi:
            continue

        if not os.path.exists(lbl_file):
            continue

        # Load the file and add scalar to the vertices
        parc_lbl = nib.freesurfer.io.read_label(lbl_file)                
        pial_scalars[parc_lbl] = roi_ptcp[roi_ix]

    # Plot the colored Brain System
    fig = mlab.figure(bgcolor=(1.0, 1.0, 1.0))
    src = mlab.pipeline.triangular_mesh_source(pial_hemi[hemi]['vert'][:,0],
                                               pial_hemi[hemi]['vert'][:,1],
                                               pial_hemi[hemi]['vert'][:,2],
                                               pial_hemi[hemi]['tria'], scalars=pial_scalars, opacity=0.75, figure=fig)
    norms = mlab.pipeline.poly_data_normals(src, figure=fig)
    norms.filter.splitting = False
    surf = mlab.pipeline.surface(norms, figure=fig)
    surf.parent.scalar_lut_manager.set(lut_mode='coolwarm', data_range=[-xmax, xmax], use_default_range=False)
    lut = surf.module_manager.scalar_lut_manager.lut.table.to_array()
    surf.module_manager.scalar_lut_manager.lut.table = lut

    # Rotate the view and save a screenshot
    pixmap = {}
    for ang in view_angle.keys():
        mlab.view(azimuth=view_angle[ang][0],
                  elevation=view_angle[ang][1])
        pixmap[ang] = mlab.screenshot(mode='rgba')
    mlab.close(all=True)

    # Save to system pixmap dictionary
    if not hemi in brain_system_pixmap.keys():
        brain_system_pixmap[hemi] = pixmap

#np.savez(pixmap_path,
#     brain_system_pixmap=brain_system_pixmap)

# Subgraph Regime Switching

In [ ]:
%matplotlib inline

coef_ix = np.array(df_cfg['cfg_obs_lut'], dtype=int)

for expr_ii, expr_id in enumerate(df_cfg['cfg_key_label'][()]['Experiment_ID']):
    
    fac_pos_neg = np.zeros((len(df_cfg['cfg_key_label'][()]['Subject_ID']), n_fac, len(df_cfg['cfg_key_label'][()]['CorSign_ID'])))        
    for ii, fac_key in enumerate(np.sort(sort_fac_dict.keys())):
        if fac_key in motion_fac:
            continue
        fac_ii = sort_fac_dict[fac_key]
        sel_fac_coef = df_subgraph['system_subgraph'][fac_ii]['expr_coef'][coef_ix] 
        sel_fac_coef = fac_coef[fac_ii, :][coef_ix]
        for sgn_ii, sgn_id in enumerate(df_cfg['cfg_key_label'][()]['CorSign_ID']):     
            lo_expr = sel_fac_coef[:, expr_ii, :, :, :, :][:, :, :, sgn_ii, :][:, 0, 1, :]
            hi_expr = sel_fac_coef[:, expr_ii, :, :, :, :][:, :, :, sgn_ii, :][:, 1, 1, :]
            
            diff_expr = (hi_expr-lo_expr) / (hi_expr+lo_expr)
            diff_expr = np.nan_to_num(diff_expr)
            
            rel_expr = np.nanmean(diff_expr, axis=-1)
            
            fac_pos_neg[:, ii, sgn_ii] = rel_expr

    plt.figure(dpi=300)
    ax = plt.subplot(111)
    ax.scatter(fac_pos_neg.mean(axis=0)[:, 0], fac_pos_neg.mean(axis=0)[:, 1]);
    print(stats.pearsonr(*fac_pos_neg.mean(axis=0).T))
    plt.show()

In [ ]:
%matplotlib inline

coef_ix = np.array(df_cfg['cfg_obs_lut'], dtype=int)

for expr_ii, expr_id in enumerate(df_cfg['cfg_key_label'][()]['Experiment_ID']):
    
    lo_task = np.zeros((len(df_cfg['cfg_key_label'][()]['Subject_ID']), n_fac-len(motion_fac), len(df_cfg['cfg_key_label'][()]['CorSign_ID'])))
    hi_task = np.zeros((len(df_cfg['cfg_key_label'][()]['Subject_ID']), n_fac-len(motion_fac), len(df_cfg['cfg_key_label'][()]['CorSign_ID'])))
    
    cnt = 0
    for ii, fac_key in enumerate(np.sort(sort_fac_dict.keys())):
        if fac_key in motion_fac:
            continue
        fac_ii = sort_fac_dict[fac_key]
        #sel_fac_coef = df_subgraph['system_subgraph'][fac_ii]['expr_coef'][coef_ix] 
        sel_fac_coef = fac_coef[fac_ii, :][coef_ix]

        for sgn_ii, sgn_id in enumerate(df_cfg['cfg_key_label'][()]['CorSign_ID']):
            lo_task[:, cnt, sgn_ii] = sel_fac_coef[:, expr_ii, :, :, :, :][:, 0, 1, :, :][:, sgn_ii, :].mean(axis=-1)
            hi_task[:, cnt, sgn_ii] = sel_fac_coef[:, expr_ii, :, :, :, :][:, 1, 1, :, :][:, sgn_ii, :].mean(axis=-1)
        
        cnt += 1
    
    
    plt.figure(dpi=300)
    ax = plt.subplot(111)
    ax.scatter(lo_task.mean(axis=0)[:, 0], lo_task.mean(axis=0)[:, 1], color='b')
    ax.scatter(hi_task.mean(axis=0)[:, 0], hi_task.mean(axis=0)[:, 1], color='r')    
    #print(stats.ttest_rel(lo_r, hi_r))
    plt.show()
    
    lo_r = [np.arctanh(stats.spearmanr(*lo_task[:, ii, :].T)[0]) for ii in xrange(n_fac-len(motion_fac))]
    hi_r = [np.arctanh(stats.spearmanr(*hi_task[:, ii, :].T)[0]) for ii in xrange(n_fac-len(motion_fac))]

    plt.boxplot([lo_r, hi_r])
    plt.show()
    print(stats.ttest_rel(lo_r, hi_r))

# Task Performance

## Behavioral Correlation

In [ ]:
%matplotlib inline

coef_ix = np.array(df_cfg['cfg_obs_lut'], dtype=int)
perf_id = 'meanRT'

for expr_ii, expr_id in enumerate(df_cfg['cfg_key_label'][()]['Experiment_ID']):
    print
    print(expr_id)
    for ii, fac_key in enumerate(np.sort(sort_fac_dict.keys())):
        #if fac_key in motion_fac:
        #    continue
        fac_ii = sort_fac_dict[fac_key]
        #sel_fac_coef = df_subgraph['system_subgraph'][fac_ii]['expr_coef'][coef_ix]
        sel_fac_coef = fac_coef[fac_ii, :][coef_ix]        
        
        pos_expr = sel_fac_coef[:, expr_ii, :, :, :, :][:, 0, 1, 0, :]
        neg_expr = sel_fac_coef[:, expr_ii, :, :, :, :][:, 0, 1, 1, :]        
        rel_expr_lo = pos_expr#-neg_expr

        pos_expr = sel_fac_coef[:, expr_ii, :, :, :, :][:, 1, 1, 0, :]
        neg_expr = sel_fac_coef[:, expr_ii, :, :, :, :][:, 1, 1, 1, :]        
        rel_expr_hi = pos_expr#-neg_expr
    
        rel_expr = rel_expr_hi-rel_expr_lo
        
        rv, pv = stats.spearmanr(rel_expr.mean(axis=-1),
                                 (df_perf[expr_id]['hi'][perf_id]-df_perf[expr_id]['lo'][perf_id]).mean(axis=-1))
        print(rv, pv)
        
        if pv < 0.05:
            plt.figure(figsize=(3,3), dpi=200.0)
            ax = plt.subplot(111)
            ax.scatter(rel_expr.mean(axis=-1),
                       (df_perf[expr_id]['hi'][perf_id]-df_perf[expr_id]['lo'][perf_id]).mean(axis=-1))
            ax.set_xlim([1.1*rel_expr.mean(axis=-1).min(), 1.1*rel_expr.mean(axis=-1).max()])
            plt.show()
        
        

In [ ]:
subgraph_beh_path = '{}/Subgraph.Behavior_Corr.npz'.format(path_ExpData)
if not os.path.exists(subgraph_beh_path):
    beh_stats_table = np.load(subgraph_beh_path)['stats_table'][()]
else:
    import statsmodels.api as sm
    import statsmodels.graphics.regressionplots as regplot

    beh_stats_table = {'Subgraph_ID': [],
                       'Task': [],
                       'Task_Cond': [],
                       'Expr_Type': [],
                       'rho': [],
                       'pval': []}
    
    for task in df['Task_Type'].unique():
        sel_task = df[df['Task_Type'] == task]
        subg_task = sel_task.groupby(['Subgraph_ID', 'Subject_ID', 'Intr_Type', 'Cond_Type']).mean().unstack()

        rs_pos = np.array(subg_task['Expression']['rs'].unstack()['pos'].unstack())
        rs_neg = np.array(subg_task['Expression']['rs'].unstack()['neg'].unstack())    
        hi_pos = np.array(subg_task['Expression']['hi'].unstack()['pos'].unstack())
        hi_neg = np.array(subg_task['Expression']['hi'].unstack()['neg'].unstack())
        lo_pos = np.array(subg_task['Expression']['lo'].unstack()['pos'].unstack())
        lo_neg = np.array(subg_task['Expression']['lo'].unstack()['neg'].unstack())

        hi_perf = subg_task['Performance']['hi'].unstack()['pos'].unstack().mean(axis=0)
        lo_perf = subg_task['Performance']['lo'].unstack()['pos'].unstack().mean(axis=0)
        cost_perf = hi_perf-lo_perf    

        prf_pairs = [('Lo-Rs', lo_perf), ('Hi-Lo', cost_perf)]
        pos_pairs = [[lo_pos, rs_pos], [hi_pos, lo_pos]]
        neg_pairs = [[lo_neg, rs_neg], [hi_neg, lo_neg]]

        print('\n\n\n\n\n*****{}*****'.format(task))
        for fac_ii in xrange(n_subgraph):
            llbl = letter_lbl[subgraph_rank == fac_ii][0]
            print(llbl)

            for prf, pos, neg in zip(prf_pairs, pos_pairs, neg_pairs):
                model1 = sm.OLS(prf[1], df_motion[task].T)
                res1 = model1.fit().resid           

                for mod_dir in [('Pos', pos),
                                ('Neg', neg)]:
                    mod_idx = mod_dir[1][0][fac_ii, :]-mod_dir[1][1][fac_ii, :]

                    model2 = sm.OLS(mod_idx, df_motion[task].T)
                    res2 = mod_idx #model2.fit().resid           

                    real_rho, real_pval = stats.spearmanr(res2, res1)

                    real_m, real_b, _, _, _ = stats.linregress(res2, res1)
                    """
                    ### Permutation Tests
                    null_rho_dist = []
                    for n_i in xrange(10000):
                        null_expr = np.random.permutation(np.array(subg_task['Expression']).reshape(-1)).reshape(n_subgraph, n_subj, 6)
                        null_expr_1 = null_expr[fac_ii, :, 0]
                        null_expr_2 = null_expr[fac_ii, :, 1]
                        mod_idx_null = null_expr_1-null_expr_2

                        model3 = sm.OLS(mod_idx_null, df_motion[task].T)
                        res3 = mod_idx_null

                        null_rho, _ = stats.spearmanr(res3, res1)
                        null_rho_dist.append(null_rho)

                    null_rho_dist = np.array(null_rho_dist)
                    if real_rho > 0:
                        real_pval = np.mean(null_rho_dist > real_rho)
                    else:
                        real_pval = np.mean(null_rho_dist < real_rho)
                    """
                    
                    if real_pval < 0.05:
                        sig = '*'
                    else:
                        sig = ' '
                                        
                    # Cache and Print output
                    beh_stats_table['Subgraph_ID'].append(fac_ii)
                    beh_stats_table['Task'].append(task)
                    beh_stats_table['Task_Cond'].append(prf[0])
                    beh_stats_table['Expr_Type'].append(mod_dir[0])
                    beh_stats_table['rho'].append(real_rho)
                    beh_stats_table['pval'].append(real_pval)
                    
                    print('%s, %s, %0.4f, %0.4f, %s' % 
                          (prf[0], mod_dir[0], real_rho, real_pval, sig))

                    """
                    # First, plot the 2D real distribution and ellipsoid errors
                    plt.figure()
                    ax = plt.subplot(111)

                    ax.scatter(res2, res1,
                               s=20, lw=0, color=[0.5, 0.5, 0.5])        


                    # Next, plot the linear fits
                    x = np.array([res2.min(), res2.max()])
                    y = real_m*x + real_b
                    ax.plot(x, y, color='k', lw=1.0)   
                    ax.text(0.75*(x.max()+x.min()),
                            0.75*(y.max()+y.min()),
                            'r=%0.3f\np=%0.3f' % (real_rho, real_pval))  

                    # Get the axis boundaries
                    x_bound = 1.05*np.max([np.abs(res2.min()),
                                           np.abs(res2.max())])
                    y_bound = 1.05*np.max([np.abs(res1.min()),
                                           np.abs(res1.max())])
                    ax.set_xlim([-x_bound, x_bound])
                    ax.set_ylim([-y_bound, y_bound])                

                    plt.savefig('{}/{}.Subgraph_{}.{}.{}.svg'.format(path_Figures, task, llbl,
                                                                     prf[0], mod_dir[0]))

                    plt.close()
                    """
    df_stats_table = pd.DataFrame(beh_stats_table, columns=beh_stats_table.keys())
    

## Regional Contribution to Modulation in Performance

In [ ]:
all_subgraph = np.array([subg['full_subg'] for subg in sys_subgraph])
all_subg_cfg = convert_adj_matr_to_cfg_matr(all_subgraph)

roi_subg_partc = np.mean(all_subgraph, axis=2)

### Surrogate ROI Participation Coefficient
"""
n_perm = 1000
null_roi_subg_partc = []
for null_ii in xrange(n_perm):
    null_subg_cfg = np.array([convert_conn_vec_to_adj_matr(subg)
                              for subg in np.random.permutation(all_subg_cfg.T).T])    
    null_roi_subg_partc.append(np.mean(null_subg_cfg, axis=2))
null_roi_subg_partc = np.array(null_roi_subg_partc)
"""

### Project Subgraph Performance to the ROI Participation Coefficient
roi_corr_partc_thrshd = {}
for task in np.unique(df_stats_table['Task']):
    for task_cond in np.unique(df_stats_table['Task_Cond']):
        sel_stats = df_stats_table.loc[(df_stats_table['Task'] == task) & 
                                       (df_stats_table['Task_Cond'] == task_cond)]

        pos_rho = sel_stats[sel_stats['Expr_Type'] == 'Pos']['rho']
        neg_rho = -1*sel_stats[sel_stats['Expr_Type'] == 'Neg']['rho']
        subg_corr = (np.array(pos_rho) + np.array(neg_rho)) / 2.0
        behpos_ix = subg_corr > 0
        behneg_ix = subg_corr < 0        
        
        for perf_type in [('BehPos', behpos_ix),
                          ('BehNeg', behneg_ix)]:
            key_name = '{}.{}.{}'.format(task, task_cond, perf_type[0])
            
            if perf_type[0] == 'BehNeg':
                sel_subg_corr = -1*subg_corr[perf_type[1]]
            else:
                sel_subg_corr = subg_corr[perf_type[1]]
                
            roi_corr_partc = np.dot(sel_subg_corr, roi_subg_partc[perf_type[1], :])

            ### Threshold using the null distribution
            null_roi_corr_partc = []
            for null_ii in xrange(n_perm):
                null_roi_corr_partc.append(np.dot(sel_subg_corr,  null_roi_subg_partc[null_ii, :, :][perf_type[1], :]))
            null_roi_corr_partc = np.array([null_roi_corr_partc]).reshape(-1)

            null_max_thr = np.percentile(null_roi_corr_partc, 100)

            roi_corr_partc_thrshd[key_name] = roi_corr_partc.copy()
            roi_corr_partc_thrshd[key_name][roi_corr_partc < null_max_thr] = np.nan
            

            # Compute the histogram bin range using the
            # real and null distribution range
            xmin = roi_corr_partc.min()
            xmax = roi_corr_partc.max()
            real_bin_range = np.linspace(xmin, xmax, 50)            
            null_xmin = null_roi_corr_partc.min()
            null_xmax = null_roi_corr_partc.max()
            null_bin_range = np.linspace(null_xmin, null_xmax, 1000)            

            ### Histogram Plots            
            fig = plt.figure(figsize=(3,3), dpi=300)
            ax = fig.add_subplot(111)
            ax.hist(roi_corr_partc, bins=real_bin_range,
                    color='r', alpha=0.3, lw=0, normed=True)
            ax.hist(null_roi_corr_partc, bins=null_bin_range,
                    color='k', alpha=0.3, lw=0, normed=True)
            ax.vlines(null_xmin, 0, 800)
            ax.vlines(null_xmax, 0, 800)            
            ax.xaxis.set_ticks_position('bottom')
            ax.yaxis.set_ticks_position('left')
            ax.set_xlim([0.0, 0.01])
            ax.set_ylim([0.0, 5000])            
            ax.set_title(key_name)
            fig.savefig('{}/ROI_Partc_Perf.{}.svg'.format(path_Figures, key_name))
            plt.close()
            
            ### Print an ordered list of the significant brain regions
            print
            print
            print('***** {} *****'.format(key_name))
            good_ix = ~np.isnan(np.sort(roi_corr_partc_thrshd[key_name])[::-1])
            sig_roi = atlas_lbl[np.argsort(roi_corr_partc_thrshd[key_name])[::-1]][good_ix] 
            sig_roi, ind = np.unique([roi.split('_')[1] for roi in sig_roi], return_index=True)
            print(sig_roi[np.argsort(ind)])

In [ ]:
from matplotlib.offsetbox import AnnotationBbox, OffsetImage, TextArea
from mayavi import mlab
import nibabel as nib


view_angle = {'Sag_PA': [0.0, 90.0],
              'Sag_AP': [180.0, 90.0]}

# Get the pial surface recons
pial_hemi = {'LH': {},
             'RH': {}}
pial_hemi['LH']['vert'], pial_hemi['LH']['tria'] = nib.freesurfer.io.read_geometry('{}/BrainRenderSubject15/surf/lh.pial'.format(path_CoreData))
pial_hemi['RH']['vert'], pial_hemi['RH']['tria'] = nib.freesurfer.io.read_geometry('{}/BrainRenderSubject15/surf/rh.pial'.format(path_CoreData))

# Get the Lausanne label files for each ROI
label_files = []
for roi in df_system['lausanne_labels']:
    hemi = roi.split('_')[0].lower()

    # Parse the atlas name and find the label file if it exists
    if len(roi.split('_')) == 2:
        lbl_file = ('%s.%s.label' % tuple(roi.split('_'))).lower()
    elif len(roi.split('_')) == 3:
        lbl_file = ('%s.%s_%s.label' % tuple(roi.split('_'))).lower()
    else:
        raise Exception
    lbl_file = lbl_file.replace(' ', '')

    label_files.append('{}/BrainRenderSubject15/label/regenerated_{}_125/{}'.format(path_CoreData, hemi, lbl_file))

    
# Iterate over experimental conditions
for corr_key in roi_corr_partc_thrshd.keys():
    xmin = np.percentile(roi_corr_partc_thrshd[corr_key][~np.isnan(roi_corr_partc_thrshd[corr_key])], 0)
    xmax = np.percentile(roi_corr_partc_thrshd[corr_key][~np.isnan(roi_corr_partc_thrshd[corr_key])], 95)
    
    pixmap = {}
    for hemi in pial_hemi.keys():
        n_vert = len(pial_hemi[hemi]['vert'])
        pial_scalars = np.zeros(n_vert)

        # Iterate over ROIs
        for roi_ix, (roi, lbl_file) in enumerate(zip(df_system['lausanne_labels'],
                                                     label_files)):
            if roi.split('_')[0] != hemi:
                continue

            if not os.path.exists(lbl_file):
                continue

            # Load the file and add scalar to the vertices
            parc_lbl = nib.freesurfer.io.read_label(lbl_file)   
            if not np.isnan(roi_corr_partc_thrshd[corr_key][roi_ix]):
                pial_scalars[parc_lbl] = roi_corr_partc_thrshd[corr_key][roi_ix]

        # Plot the colored Brain System
        fig = mlab.figure(bgcolor=(1.0, 1.0, 1.0))
        src = mlab.pipeline.triangular_mesh_source(pial_hemi[hemi]['vert'][:,0],
                                                   pial_hemi[hemi]['vert'][:,1],
                                                   pial_hemi[hemi]['vert'][:,2],
                                                   pial_hemi[hemi]['tria'], scalars=pial_scalars, opacity=0.45, figure=fig)
        norms = mlab.pipeline.poly_data_normals(src, figure=fig)
        norms.filter.splitting = False
        surf = mlab.pipeline.surface(norms, figure=fig)
        if corr_key.split('.')[-1] == 'BehNeg':
            cmap = 'Blues'
        else:
            cmap = 'Reds'
        surf.parent.scalar_lut_manager.set(lut_mode=cmap, data_range=[xmin, xmax], use_default_range=False)
        lut = surf.module_manager.scalar_lut_manager.lut.table.to_array()
        surf.module_manager.scalar_lut_manager.lut.table = lut

        # Rotate the view and save a screenshot
        for ang in view_angle.keys():
            mlab.view(azimuth=view_angle[ang][0],
                      elevation=view_angle[ang][1])
            pixmap['{}_{}'.format(hemi, ang)] = mlab.screenshot(mode='rgba')
        mlab.close(all=True)
    
    ### Plot the pixmap
    fig = plt.figure(figsize=(2,2), dpi=300)
    ax = fig.add_subplot(111)

    pixmap_key = [('LH_Sag_AP', False, (0.25, 0.575)),
                  ('RH_Sag_PA', False, (0.75, 0.575)),
                  ('LH_Sag_PA', True, (0.25, 0.225)),
                  ('RH_Sag_AP', True, (0.75, 0.225))]

    for px_key in pixmap_key:
        if px_key[1] == True:
            imagebox = OffsetImage(pixmap[px_key[0]][:, ::-1, :], zoom=0.65)
        else:
            imagebox = OffsetImage(pixmap[px_key[0]], zoom=0.65)
        imagebox.axes = fig.axes[0]
        ab = AnnotationBbox(imagebox, 
                            xy=px_key[2],
                            xycoords='data',
                            frameon=False,
                            pad=0.0)
        fig.axes[0].add_artist(ab)
    ax.set_axis_off()
    ax.set_title('{}\n{}'.format(corr_key, xmax))
    fig.savefig('{}/ROI_Partc_Perf.Atlas.{}.svg'.format(path_Figures, corr_key))
    fig.show()

In [ ]:
plt.figure()
ax = plt.subplot(211)
mat = ax.imshow(pixmap[px_key[0]], cmap='Blues')
ax.set_axis_off()
plt.colorbar(mat, ax=ax)

ax = plt.subplot(221)
mat = ax.imshow(pixmap[px_key[0]], cmap='Reds')
ax.set_axis_off()
plt.colorbar(mat, ax=ax)

plt.savefig('{}/Blues_Reds_CB.svg'.format(path_Figures))
plt.show()